# <B> Model Deployment </B>
* Container: codna_pytorch_py39

## AutoReload

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. parameter store 설정

In [3]:
import boto3
from utils.ssm import parameter_store

In [4]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
prefix = pm.get_params(key="PREFIX")

## 2.package import for model deployment

In [5]:
import os
import sagemaker
from sagemaker.pytorch.model import PyTorchModel

In [6]:
from time import strftime
from smexperiments.trial import Trial
from smexperiments.experiment import Experiment

In [7]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name)

In [8]:
def create_trial(experiment_name):
    create_date = strftime("%m%d-%H%M%s")
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

In [10]:
local_mode = False

if local_mode: 
    inference_instance_type = 'local_gpu'
    
    import os
    from sagemaker.local import LocalSession
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    
else:
    inference_instance_type = "ml.g4dn.xlarge"
    sagemaker_session = sagemaker.Session()
    


sagemaker_role_arn = pm.get_params(key=prefix + '-SAGEMAKER-ROLE-ARN')    
bucket_name = pm.get_params(key=prefix + '-BUCKET')
model_artifact_s3_uri = pm.get_params(key=prefix + '-MODEL-PATH')
inf_image_uri = pm.get_params(key=''.join([prefix, '-INF-IMAGE-URI']))
#inf_image_uri = pm.get_params(key=''.join([prefix, '-IMAGE-URI']))

code_location= os.path.join(
    "s3://{}".format(bucket_name),
    prefix,
    "inference",
    "backup_codes"
)

monitor_output= os.path.join(
    "s3://{}".format(bucket_name),
    prefix,
    "inference",
    "monitor_output"
)

print (f"sagemaker_role_arn: {sagemaker_role_arn}")
print (f"model_artifact_s3_uri: {model_artifact_s3_uri}")
print (f"inf_image_uri: {inf_image_uri}")
print (f"code_location: {code_location}")
print (f"monitor_output: {monitor_output}")

sagemaker_role_arn: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
model_artifact_s3_uri: s3://sm-nemo-bucket/nemo-asr/training/model-output/nemo-asr-nemo-experiments-0320-09481679305686/output/model.tar.gz
inf_image_uri: 419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-training:latest
code_location: s3://sm-nemo-bucket/nemo-asr/inference/backup_codes
monitor_output: s3://sm-nemo-bucket/nemo-asr/inference/monitor_output


* Define inference job

In [11]:
model = PyTorchModel(
    entry_point='predictor.py',
    source_dir=os.getcwd() + '/code',
    code_location=code_location,
    model_data=model_artifact_s3_uri,
    role=sagemaker_role_arn,
    image_uri=inf_image_uri,
    # framework_version="1.13.1",
    # py_version="py39",
    sagemaker_session=sagemaker_session
)

In [12]:
if local_mode: 
    data_capture_config = None
else:
    from sagemaker.model_monitor import DataCaptureConfig

    data_capture_config = DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=monitor_output
    )

In [ ]:
experiment_name = pm.get_params(key=prefix + "-EXPERI-NAME")
create_experiment(experiment_name)
job_name = create_trial(experiment_name)


predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    data_capture_config=data_capture_config,
    endpoint_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    }
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating model with name: nemo-test-training-2023-03-21-02-11-23-416
INFO:sagemaker:Creating endpoint-config with name nemo-asr-nemo-experiments-0321-02101679364642
INFO:sagemaker:Creating endpoint with name nemo-asr-nemo-experiments-0321-02101679364642


----------

In [ ]:
paths2audio_files = f"{os.getcwd()}/data/preprocessing/an4/wav/an4test_clstk/fcaw/an406-fcaw-b.wav"
paths2audio_files

In [ ]:
import librosa
import IPython.display as ipd

# Load and listen to the audio file
audio, sample_rate = librosa.load(paths2audio_files)

ipd.Audio(paths2audio_files, rate=sample_rate)

In [ ]:
from sagemaker.serializers import DataSerializer
predictor.serializer = DataSerializer()

In [ ]:
predictor.predict(paths2audio_files)

In [ ]:
pm.put_params(key="-".join([prefix, "ENDPOINT-NAME"]), value=job_name, overwrite=True)
pm.put_params(key="-".join([prefix, "MONITOR-OUTPUT"]), value=monitor_output, overwrite=True)

In [ ]:
# predictor.delete_endpoint()